# NYC Flights 2013 Analysis

In [9]:
install.packages("nycflights13")
library(nycflights13)
library("tidyverse")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [10]:
glimpse(flights)
cat("\n")
glimpse(airlines)
cat("\n")
glimpse(airports)
cat("\n")
glimpse(weather)
cat("\n")
glimpse(planes)
cat("\n")

Rows: 336,776
Columns: 19
$ year           <int> 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2…
$ month          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ day            <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ dep_time       <int> 517, 533, 542, 544, 554, 554, 555, 557, 557, 558, 558, …
$ sched_dep_time <int> 515, 529, 540, 545, 600, 558, 600, 600, 600, 600, 600, …
$ dep_delay      <dbl> 2, 4, 2, -1, -6, -4, -5, -3, -3, -2, -2, -2, -2, -2, -1…
$ arr_time       <int> 830, 850, 923, 1004, 812, 740, 913, 709, 838, 753, 849,…
$ sched_arr_time <int> 819, 830, 850, 1022, 837, 728, 854, 723, 846, 745, 851,…
$ arr_delay      <dbl> 11, 20, 33, -18, -25, 12, 19, -14, -8, 8, -2, -3, 7, -1…
$ carrier        <chr> "UA", "UA", "AA", "B6", "DL", "UA", "B6", "EV", "B6", "…
$ flight         <int> 1545, 1714, 1141, 725, 461, 1696, 507, 5708, 79, 301, 4…
$ tailnum        <chr> "N14228", "N24211", "N619AA", "N804JB", "N668DN", "N394…
$ origin      

## Q1: Which airlines had the least departure delay at the beginning of 2013?

In [11]:
flights %>%
  filter(month == 1, day == 1) %>%
  select(carrier, dep_delay) %>%
  arrange(dep_delay) %>%
  head(5) %>%
  inner_join(airlines, by = "carrier")

carrier,dep_delay,name
<chr>,<dbl>,<chr>
MQ,-15,Envoy Air
AA,-15,American Airlines Inc.
F9,-14,Frontier Airlines Inc.
EV,-13,ExpressJet Airlines Inc.
B6,-12,JetBlue Airways


## Q2: Which flights go more than 50 days of the year and where do they fly to?

In [12]:
flights %>% 
  group_by(carrier, flight, dest) %>%
  summarise(n=n(), .groups = 'drop') %>%
  arrange(desc(n)) %>% 
  filter(n > 49) %>%
  head(5)

carrier,flight,dest,n
<chr>,<int>,<chr>,<int>
AA,59,SFO,365
AA,119,LAX,365
AA,181,LAX,365
AA,1357,SJU,365
AA,1611,MIA,365


## Q3: Which destinations did people travel to the most during the summer (July, August and September) of 2013?

In [13]:
flights %>%
  filter(!is.na(dep_time), year == 2013, month %in% c(7,8,9),) %>%
  select(year, month, dest) %>%
  group_by(year, month, dest) %>% 
  summarise(n_filghts = n(), .groups = 'drop') %>%
  arrange(desc(n_filghts)) %>%
  head(5) %>%
  inner_join(airports %>%
               select(faa, name),
             by = c("dest" = "faa"))

year,month,dest,n_filghts,name
<int>,<int>,<chr>,<int>,<chr>
2013,8,ORD,1571,Chicago Ohare Intl
2013,9,ORD,1517,Chicago Ohare Intl
2013,8,LAX,1502,Los Angeles Intl
2013,7,ORD,1499,Chicago Ohare Intl
2013,7,LAX,1497,Los Angeles Intl


## Q4: Which plane (tailnum) has the worst on-time record?

In [14]:
flights %>%
  filter(!is.na(tailnum)) %>%
  mutate(on_time = !is.na(arr_time) & (arr_delay <= 0)) %>%
  group_by(tailnum) %>%
  summarise(on_time = mean(on_time), n = n()) %>%
  filter(min_rank(on_time) == 1) %>%
  head(5)

tailnum,on_time,n
<chr>,<dbl>,<int>
N121DE,0,2
N136DL,0,1
N143DA,0,1
N17627,0,2
N240AT,0,5


## Q5: Which airlines had the most canceled flights in 2013?

In [15]:
deptime <- flights %>%                                   
  filter(is.na(dep_time)) %>%                               
  group_by(carrier) %>%                                     
  summarise(canceled_flights = n())                         

flights %>% 
  inner_join(airlines, by = "carrier") %>%                  
  group_by(carrier, name) %>% 
  select(carrier, name) %>%                                 
  summarise(scheduled_flights = n(), .groups = 'drop') %>%          
  inner_join(deptime, by = "carrier") %>%                
  mutate(canceled_flights_proportion =                      
           round((canceled_flights/scheduled_flights), 3)) %>% 
  arrange(desc(canceled_flights_proportion)) %>%            
  head(5) 

carrier,name,scheduled_flights,canceled_flights,canceled_flights_proportion
<chr>,<chr>,<int>,<int>,<dbl>
OO,SkyWest Airlines Inc.,32,3,0.094
YV,Mesa Airlines Inc.,601,56,0.093
9E,Endeavor Air Inc.,18460,1044,0.057
EV,ExpressJet Airlines Inc.,54173,2817,0.052
MQ,Envoy Air,26397,1234,0.047
